# Auto-Batched Joint Distributions: A Gentle Tutorial

##### Copyright 2020 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/probability/examples/Modeling_with_JointDistribution"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/probability/blob/main/tensorflow_probability/examples/jupyter_notebooks/Modeling_with_JointDistribution.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/jupyter_notebooks/Modeling_with_JointDistribution.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/probability/tensorflow_probability/examples/jupyter_notebooks/Modeling_with_JointDistribution.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

### 시작하기

TensorFlow Probability(TFP)는 여러 `JointDistribution` 추상화를 제공함으로 사용자가 수학적 형식에 가까운 확률적 그래프 모델을 쉽게 표현할 수 있도록 하여 확률적 추론을 더 쉽게 만듭니다. 추상화는 모델에서 샘플링하고 모델에서 샘플의 로그 확률을 평가하는 방법을 생성합니다. 이 튜토리얼에서는 원본 `JointDistribution` 추상화 이후 개발된 '자동 일괄 처리된' 변형을 검토합니다. 원래의 자동 일괄 처리되지 않은 추상화에 비해 자동 일괄 처리 버전은 사용이 더 간단하고 인체 공학적이므로 더 적은 상용구로 많은 모델을 표현할 수 있습니다. 이 colab에서는 단순한 모델을 자세히 탐색하여(다소 지루할 수 있음) 자동 일괄 처리로 해결되는 문제를 명확하게 하고, 앞으로 독자에게 TFP 형상 개념을 더 많이 소개할 수 있기를 바랍니다.

Prior to the introduction of autobatching, there were a few different variants of `JointDistribution`, corresponding to different syntactic styles for expressing probabilistic models: `JointDistributionSequential`, `JointDistributionNamed`, and`JointDistributionCoroutine`. Auobatching exists as a mixin, so we now have `AutoBatched` variants of all of these. In this tutorial, we explore the differences between `JointDistributionSequential` and `JointDistributionSequentialAutoBatched`; however, everything we do here is applicable to the other variants with essentially no changes.


### 종속성과 전제 조건


In [ ]:
#@title Import and set ups{ display-mode: "form" }

import functools
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_probability as tfp

tfd = tfp.distributions

### 전제 조건: 베이지안 회귀 문제

We'll consider a very simple Bayesian regression scenario:

$$ \begin{align*} m &amp; \sim \text{Normal}(0, 1) \ b &amp; \sim \text{Normal}(0, 1) \ Y &amp; \sim \text{Normal}(mX + b, 1) \end{align*} $$

이 모델에서 `m`과 `b`는 표준 정규 분포에서 추출되고 관측치 `Y`는 평균이 확률 변수 `m` 및 `b`에 의존하는 정규 분포와 일부 공변량(무작위가 아닌, 알려진 공변량) `X`에서 추출됩니다(간단함을 위해 이 예에서는 모든 확률 변수의 규모가 알려져 있다고 가정함).

이 모델에서 추론을 수행하려면 공변량 `X`와 관측치 `Y`를 모두 알아야 하지만, 이 튜토리얼에서는 `X`만 필요하므로 간단한 더미 `X`를 정의합니다.

In [ ]:
X = np.arange(7)
X

array([0, 1, 2, 3, 4, 5, 6])

### 데시데라타(Desiderata)

확률적 추론에서는 종종 두 가지 기본 연산을 수행하려고 합니다.

- `sample`: 모델에서 샘플을 추출합니다.
- `log_prob`: 모델에서 샘플의 로그 확률을 계산합니다.

TFP의 `JointDistribution` 추상화(확률적 프로그래밍에 대한 다른 많은 접근 방식 포함)를 사용할 때의 주요 이점은 사용자가 모델을 *한 번* 작성하고 `sample` 및 `log_prob` 계산에 모두 액세스할 수 있다는 것입니다.

데이터세트(`X.shape = (7,)`)에 7개의 지점이 있다는 점에 주목하여 이제 우수한 `JointDistribution`에 대한 데시데라타를 명시할 수 있습니다.

- `sample()`은 각각 스칼라 기울기, 스칼라 바이어스 및 벡터 관측치에 해당하는 형상 `[(), (), (7,)`]을 가진 `Tensors` 목록을 생성해야 합니다.
- `log_prob(sample())`은 특정 기울기, 바이어스 및 관측치의 로그 확률인 스칼라를 생성해야 합니다.
- `sample([5, 3])`은 모델의 샘플 `(5, 3)`-<em>배치</em>를 나타내는 형상 <code>[(5, 3), (5, 3), (5, 3, 7)]</code>을 가진 `Tensors` 목록을 생성해야 합니다.
- `log_prob(sample([5, 3]))`은 형상 (5, 3)의 `Tensor`를 생성해야 합니다.

We'll now look at a succession of `JointDistribution` models, see how to achieve the above desiderata, and hopefully learn a little more about TFP shapes along the way.

스포일러 경고: 상용구를 추가하지 않고 위의 데시데라타를 충족하는 접근 방식은 [자동 일괄 처리](#scrollTo=_h7sJ2bkfOS7)입니다. 

### 첫 번째 시도: `JointDistributionSequential`

In [ ]:
jds = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

첫 번째 시도는 모델을 코드로 직접 변환하는 것입니다. 기울기 `m`과 바이어스 `b`는 간단합니다. `Y`는 `lambda` 함수를 사용하여 정의됩니다. 일반적인 패턴은 `JointDistributionSequential`(JDS)에서 $k$ 인수의 `lambda` 함수가 모델의 이전 $k$ 분포를 사용한다는 것입니다. '역'순서에 유의하세요.

샘플을 생성하는 데 사용된 샘플 *및* 기본 '하위 분포'를 모두 반환하는 `sample_distributions`를 호출합니다(`sample`을 호출하여 샘플만 생성할 수 있습니다. 튜토리얼의 뒷부분에서 이들 분포가 있으면 편리할 것입니다). 적절한 샘플은 다음과 같습니다.

In [ ]:
dists, sample = jds.sample_distributions()
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

하지만 `log_prob`는 원하지 않는 형상이 있는 결과를 생성합니다.

In [ ]:
jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([-4.4777603, -4.6775575, -4.7430477, -4.647725 , -4.5746684,
       -4.4368567, -4.480562 ], dtype=float32)>

여러 샘플 추출하기도 동작하지 않습니다.

In [ ]:
try:
  jds.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


무엇이 잘못되었는지 이해하려고 해봅시다.

### 간략한 검토: 배치 형상 및 이벤트 형상

In TFP, an ordinary (not a `JointDistribution`) probability distribution has an *event shape* and a *batch shape*, and understanding the difference is crucial to effective use of TFP:

- 이벤트 형상은 분포에서 단일 추출의 형상을 설명합니다. 추출은 차원에 따라 달라질 수 있습니다. 스칼라 분포의 경우 이벤트 형상은 []입니다. 5차원 MultivariateNormal의 경우 이벤트 형상은 [5]입니다.
- 배치 형상은 동일하게 분포되지 않은 독립적인 추출, 일명 '배치'를 나타냅니다. 단일 Python 객체에서 분포의 배치를 나타내는 것은 TFP가 대규모 효율성을 달성하는 주요 방법 중 하나입니다.

For our purposes, a critical fact to keep in mind is that if we call `log_prob` on a single sample from a distribution, the result will always have a shape that matches (i.e., has as rightmost dimensions) the *batch* shape.

형상에 대한 자세한 내용은 ['TensorFlow 분포 형상 이해하기' 튜토리얼](https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes)을 참조하세요.


### `log_prob(sample())`이 스칼라를 생성하지 않는 이유는 무엇일까요? 

배치 형상 및 이벤트 형상에 대한 지식을 사용하여 `log_prob(sample())`에서 무슨 일이 일어나고 있는지 살펴보겠습니다. 다음의 샘플을 다시 보겠습니다.

In [ ]:
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

그리고 분포는 다음과 같습니다.

In [ ]:
dists

[<tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>]

The log probability is computed by summing the log probabilities of the sub-distributions at the (matched) elements of the parts:

In [ ]:
log_prob_parts = [dist.log_prob(s) for (dist, s) in zip(dists, sample)]
log_prob_parts

[<tf.Tensor: shape=(), dtype=float32, numpy=-2.3113134>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-1.1357536>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([-1.0306933, -1.2304904, -1.2959809, -1.200658 , -1.1276014,
        -0.9897899, -1.0334952], dtype=float32)>]

In [ ]:
np.sum(log_prob_parts) - jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

따라서 한 가지 설명으로는 `log_prob_parts`의 3번째 서브 구성 요소가 7-텐서이므로 로그 확률 계산이 7-텐서를 반환한다는 것입니다. 이유는 무엇일까요?

수학 공식에서 `Y`에 대한 분포에 해당하는 `dists`의 마지막 요소가 `[7]`의 `batch_shape`를 가짐을 확인합니다. 즉, `Y`에 대한 분포는 7개의 독립적인 정규 분포(평균이 다르며 이 경우 규모는 같음)입니다.

We now understand what's wrong: in JDS, the distribution over `Y` has `batch_shape=[7]`, a sample from the JDS represents scalars for `m` and `b` and a "batch" of 7 independent normals. and `log_prob` computes 7 separate log-probabilities, each of which represents the log probability of drawing `m` and `b` and a single observation `Y[i]` at some `X[i]`.

### `Independent`로 `log_prob(sample())` 수정하기

`dists[2]`에는 `event_shape=[]`과 `batch_shape=[7]`이 있음을 상기하세요.

In [ ]:
dists[2]

<tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>

배치 차원을 이벤트 차원으로 변환하는 TFP의 `Independent` 메타분포를 사용하여 `event_shape=[7]` 및 `batch_shape=[]`의 분포로 변환할 수 있습니다(`Y`의 분포이므로 `y_dist_i` 이름을 변경하고 `_i`는 `Independent` 래핑을 대신합니다). 

In [ ]:
y_dist_i = tfd.Independent(dists[2], reinterpreted_batch_ndims=1)
y_dist_i

<tfp.distributions.Independent 'IndependentJointDistributionSequential_sample_distributions_Normal' batch_shape=[] event_shape=[7] dtype=float32>

이제 7-벡터의 `log_prob`는 스칼라입니다.

In [ ]:
y_dist_i.log_prob(sample[2])

<tf.Tensor: shape=(), dtype=float32, numpy=-7.9087086>

Under the covers, `Independent` sums over the batch:

In [ ]:
y_dist_i.log_prob(sample[2]) - tf.reduce_sum(dists[2].log_prob(sample[2]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

그리고 실제로 이를 사용하여 `log_prob`가 스칼라를 반환하는 새로운 `jds_i`(`i`는 다시 `Independent` 나타냄)를 생성할 수 있습니다.

In [ ]:
jds_i = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m*X + b, scale=1.),
        reinterpreted_batch_ndims=1)
])

jds_i.log_prob(sample)

<tf.Tensor: shape=(), dtype=float32, numpy=-11.355776>

몇 가지 참고 사항입니다.

- `jds_i.log_prob(s)`은 `tf.reduce_sum(jds.log_prob(s))`와 같지 *않*습니다. 전자는 결합 분포의 '올바른' 로그 확률을 생성합니다. 후자는 7-텐서에 대해 합하고, 각 요소는 `m`, `b` 및 `Y` 로그 확률의 단일 요소의 합이므로 `m`과 `b`를 초과합니다(`log_prob(m) + log_prob(b) + log_prob(Y)`는 TFP가 TF 및 NumPy의 브로드캐스팅 규칙을 따르므로 예외로 처리하지 않고 결과를 반환합니다. 벡터에 스칼라를 추가하면 벡터 크기의 결과가 생성됩니다).
- 이 특정 경우에는 문제를 해결하고 `Independent(Normal(...))` 대신 `MultivariateNormalDiag`를 사용하여 같은 결과를 얻을 수 있습니다. `MultivariateNormalDiag`는 벡터 값 분포입니다(즉, 이미 벡터 이벤트 형상이 있음). `MultivariateNormalDiag`는 `Independent`와 `Normal`의 구성으로 구현될 수 있지만 실제로 구현되지는 않습니다. 벡터 `V`가 주어지면 `n1 = Normal(loc=V)`와 `n2 = MultivariateNormalDiag(loc=V)`의 샘플은 구별할 수 없음을 기억하는 것이 좋습니다. 이러한 분포의 차이점은 `n1.log_prob(n1.sample())`이 벡터이고 `n2.log_prob(n2.sample())`은 스칼라라는 것입니다.

### 여러 샘플

여러 샘플 추출하기가 여전히 동작하지 않습니다.

In [ ]:
try:
  jds_i.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


그 이유를 생각해봅시다. `jds_i.sample([5, 3])`을 호출할 때 먼저 `m`과 `b` 샘플을 각각 형상 `(5, 3)`으로 추출합니다. 그 후, 다음을 통해 `Normal` 분포를 구성하려고 합니다.

```
tfd.Normal(loc=m*X + b, scale=1.)
```

그러나 `m`이 형상 `(5, 3)`이고 `X`가 형상 `7`이면 이 둘을 함께 곱할 수 없으며 실제로 이러한 오류가 발생합니다.

In [ ]:
m = tfd.Normal(0., 1.).sample([5, 3])
try:
  m * X
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


이 문제를 해결하기 위해 `Y`에 대한 분포에 어떤 속성이 있어야 하는지 생각해 보겠습니다. `jds_i.sample([5, 3])`을 호출했다면 `m`과 `b`가 모두 형상`(5, 3)`을 가질 것임을 압니다. `Y` 분포에서 `sample`에 대한 호출은 어떤 형상을 생성해야 할까요? 분명한 대답은 `(5, 3, 7)`입니다. 각 배치 지점에 대해 `X`와 같은 크기의 샘플이 필요합니다. TensorFlow의 브로드캐스팅 기능으로 추가 차원을 더하여 이를 달성할 수 있습니다.

In [ ]:
m[..., tf.newaxis].shape

TensorShape([5, 3, 1])

In [ ]:
(m[..., tf.newaxis] * X).shape

TensorShape([5, 3, 7])

`m`과 `b` 모두에 축을 추가하면 여러 샘플을 지원하는 새 JDS를 정의할 수 있습니다.

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

shaped_sample = jds_ia.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-1.1133379 ,  0.16390413, -0.24177533],
        [-1.1312429 , -0.6224666 , -1.8182136 ],
        [-0.31343174, -0.32932565,  0.5164407 ],
        [-0.0119963 , -0.9079621 ,  2.3655841 ],
        [-0.26293617,  0.8229698 ,  0.31098196]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-0.02876974,  1.0872147 ,  1.0138507 ],
        [ 0.27367726, -1.331534  , -0.09084719],
        [ 1.3349475 , -0.68765205,  1.680652  ],
        [ 0.75436825,  1.3050154 , -0.9415123 ],
        [-1.2502679 , -0.25730947,  0.74611956]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 7), dtype=float32, numpy=
 array([[[-1.8258233e+00, -3.0641669e-01, -2.7595463e+00, -1.6952467e+00,
          -4.8197951e+00, -5.2986512e+00, -6.6931367e+00],
         [ 3.6438566e-01,  1.0067395e+00,  1.4542470e+00,  8.1155670e-01,
           1.8868095e+00,  2.3877139e+00,  1.0195159e+00],
         [-8.3624744e-01,  1.2518480e+00,  1.0943471e+00, 

In [ ]:
jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.483114 , -10.139662 , -11.514159 ],
       [-11.656767 , -17.201958 , -12.132455 ],
       [-17.838818 ,  -9.474525 , -11.24898  ],
       [-13.95219  , -12.490049 , -17.123957 ],
       [-14.487818 , -11.3755455, -10.576363 ]], dtype=float32)>

추가 검사로 단일 배치 지점에 대한 로그 확률이 이전과 일치하는지 확인합니다.

In [ ]:
(jds_ia.log_prob(shaped_sample)[3, 1] -
 jds_i.log_prob([shaped_sample[0][3, 1],
                 shaped_sample[1][3, 1],
                 shaped_sample[2][3, 1, :]]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

<a id="AutoBatching-For-The-Win"></a>

### 성공적으로 자동 일괄 처리하기


아주 좋습니다. 이제 모든 데시데라타를 처리하는 JointDistribution의 버전을 갖추었습니다. `log_prob`는 `tfd.Independent`를 사용하여 스칼라를 반환하며, 추가 축을 더하여 브로드캐스팅을 수정했으므로 이제 여러 샘플이 제대로 동작합니다.

더 쉽고 더 좋은 방법이 있다면 어떨까요? 그 방법은 바로 `JointDistributionSequentialAutoBatched`(JDSAB)라고 합니다.

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

In [ ]:
jds_ab.log_prob(jds.sample())

<tf.Tensor: shape=(), dtype=float32, numpy=-12.954952>

In [ ]:
shaped_sample = jds_ab.sample([5, 3])
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.191533 , -10.43885  , -16.371655 ],
       [-13.292994 , -11.97949  , -16.788685 ],
       [-15.987699 , -13.435732 , -10.6029   ],
       [-10.184758 , -11.969714 , -14.275676 ],
       [-12.740775 , -11.5654125, -12.990162 ]], dtype=float32)>

In [ ]:
jds_ab.log_prob(shaped_sample) - jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

어떻게 동작하나요? 깊은 이해를 위해 [코드 읽기](https://github.com/tensorflow/probability/blob/main/tensorflow_probability/python/distributions/joint_distribution_auto_batched.py#L426)를 시도할 수 있지만 대부분의 사용 사례에 대해 충분한, 간략한 개요를 제공합니다.

- 첫 번째 문제는 `Y`에 `batch_shape=[7]`와 `event_shape=[]`가 있고`Independent`를 사용하여 배치 차원을 이벤트 차원으로 변환했다는 점을 기억하세요. JDSAB는 구성 요소 분포의 배치 형상을 무시합니다. 대신 `batch_ndims > 0`을 설정하여 달리 지정하지 않는 한, 배치 형상을 모델의 전체 속성으로 처리하며 `[]`로 간주합니다. 이 효과는 위에서 수동으로 수행한 것처럼 tfd.Independent를 사용하여 구성 요소 분포의 <em>모든</em> 배치 차원을 이벤트 차원으로 변환하는 것과 같습니다.
- 두 번째 문제는 여러 샘플을 만들 때 `X`로 적절하게 브로드캐스팅할 수 있도록 `m`과 `b`의 형상을 조정해야 한다는 것이었습니다. JDSAB를 사용하면 모델을 작성하여 단일 샘플을 생성하고 전체 모델을 '리프트(lift)'하여 TensorFlow의 [vectorized_map](https://www.tensorflow.org/api_docs/python/tf/vectorized_map)으로 여러 샘플을 생성합니다(이 특성은 JAX의 [vmap](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#Auto-vectorization-with-vmap)과 유사합니다).

배치 형상 문제를 더 자세히 살펴보면, 원래의 '불량' 결합 분포 `jds`, 배치 고정 분포 `jds_i`, ` jds_ia` 및 자동 일괄 처리된 `jds_ab`의 배치 형상을 비교할 수 있습니다.

In [ ]:
jds.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([7])]

In [ ]:
jds_i.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ia.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ab.batch_shape

TensorShape([])

We see that the original `jds` has subdistributions with different batch shapes. `jds_i` and `jds_ia` fix this by creating subdistributions with the same (empty) batch shape. `jds_ab` has only a single (empty) batch shape.

It's worth noting that `JointDistributionSequentialAutoBatched` offers some additional generality for free. Suppose we make the covariates `X` (and, implicitly, the observations `Y`) two-dimensional:

In [ ]:
X = np.arange(14).reshape((2, 7))
X

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13]])

Our `JointDistributionSequentialAutoBatched` works with no changes (we need to redefine the model because the shape of `X` is cached by `jds_ab.log_prob`):

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

shaped_sample = jds_ab.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.1813647 , -0.85994506,  0.27593774],
        [-0.73323774,  1.1153806 ,  0.8841938 ],
        [ 0.5127983 , -0.29271227,  0.63733214],
        [ 0.2362284 , -0.919168  ,  1.6648189 ],
        [ 0.26317367,  0.73077047,  2.5395133 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.09636458,  2.0138032 , -0.5054413 ],
        [ 0.63941646, -1.0785882 , -0.6442188 ],
        [ 1.2310615 , -0.3293852 ,  0.77637213],
        [ 1.2115169 , -0.98906034, -0.07816773],
        [-1.1318136 ,  0.510014  ,  1.036522  ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 2, 7), dtype=float32, numpy=
 array([[[[-1.9685398e+00, -1.6832136e+00, -6.9127172e-01,
            8.5992378e-01, -5.3123581e-01,  3.1584005e+00,
            2.9044402e+00],
          [-2.5645006e-01,  3.1554163e-01,  3.1186538e+00,
            1.4272424e+00,  1.2843871e+00,  1.2266440e+00,
            1.2798605e+00]],
 
         [[ 1.5973477e+00,

In [ ]:
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-28.90071 , -23.052422, -19.851362],
       [-19.775568, -25.894997, -20.302256],
       [-21.10754 , -23.667885, -20.973007],
       [-19.249458, -20.87892 , -20.573763],
       [-22.351208, -25.457762, -24.648403]], dtype=float32)>

반면에 신중하게 만들어진 `JointDistributionSequential`은 더 이상 동작하지 않습니다.

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

try:
  jds_ia.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3,1] vs. [2,7] [Op:Mul]


이 문제를 해결하려면 두 번째 `tf.newaxis`를 `m`과 `b`가 형상과 일치하도록 추가하고 `reinterpreted_batch_ndims`를 `Independent`에 대한 호출에서 2로 늘려야 합니다. 이 경우 자동 일괄 처리 기계가 형상 문제를 처리하도록 하는 것이 더 짧고 더 쉽고 인체 공학적입니다.

다시 한 번 말하자면, 이 노트북에서 `JointDistributionSequentialAutoBatched`를 탐색하는 동안 `JointDistribution`의 다른 변형에는 동등한 `AutoBatched`가 있습니다(`JointDistributionCoroutine` 사용자에게는 `JointDistributionCoroutineAutoBatched`에 더는 `Root` 노드를 지정할 필요가 없다는 추가 이점이 있습니다. `JointDistributionCoroutine`을 사용한 적이 없으면 이 설명은 생략해도 됩니다).

### 결론

이 노트북에서는 `JointDistributionSequentialAutoBatched`를 소개하고 간단한 예제를 자세히 살펴보았습니다. TFP 형상과 자동 일괄 처리에 대해 배웠기를 바랍니다.